In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from opening_hours import OpeningHours
import re

In [2]:
df = pd.read_csv('osm_places_amenity.csv')

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16888\2753785423.py:1: DtypeWarning: Columns (47,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('osm_places_amenity.csv')


In [3]:
original_hours = df['opening_hours'].astype(str).unique()

In [4]:
# Split by ;
hours = [hour.strip() for hrs in original_hours for hour in hrs.split(';')]

In [5]:
# For now let's just look at unique statements
hours = list(set(hours))

In [6]:
# Change shortenings of some days for the OpeningHours parser to work.
day_map = {
    "Mo":"Mon",
    "Tu":"Tue",
    "We":"Wed",
    "Fr": "Fri"
}

string = 'Mo-Th 12:00-22:00'
for i, _ in enumerate(hours):
    for key, val in day_map.items():
        hours[i] = hours[i].replace(key, val)

In [7]:
# Split by commas
hours = [[hr.strip()  for hr in hrs.split(',')] for hrs in hours]

In [8]:
print(hours)

[['Sa 07:30-15:00'], ['Sa 15:00-04:00'], ['Su-Mon 17:00-24:00'], ['Mon-Fri 12:00-23:30'], ['Th-Su 12:00-23:00'], ['Fri-Sa 10:00-02:15'], ['Su 17:00-01:00'], ['Mon-Su 17:00-24:00+'], ['Tue-Su 09:00-18:00'], ['Mon-Fri 17:00-2:00'], ['Su 16:00-21:30'], ['Fri-Sa 09:00-13:00'], ['Fri-Sa 12:00-02:00'], ['Fri 07:00-15:30'], ['Fri', 'Sa 11:30-15:00', '16:30-23:00'], ['Su 11:00-15:00', '05:30-23:00'], ['Mon 17:00-21:00'], ['Sa 10:00-23:00'], ['Fri 05:30-22:00'], ['Fri-Sa 10:00-23:30'], ['Mon', 'Wed', 'Th 11:15-22:15'], ['Su', 'Sa 08:00-20:00'], ['Mon-Sa 14:00-04:00'], ['Fri-Sa 17:00-01:00'], ['Su-Th 10:00-03:00'], ['Mon-Th 14:00-23:00'], ['Tue-Sa 17:00-21:00'], ['Tue-Sa 12:00-21:00'], ['Mon-Th 10:30-23:00'], ['Sa 08:30-16:00'], ['Wed 16:30-01:00'], ['Tue-Wed 07:30-23:00'], ['Fri 11:00-16:00'], ['Mon-Su 10:00-16:00'], ['Dec 31 06:00-20:00'], ['Mon-Th 13:00-24:00'], ['Su 14:00-19:00'], ['Fri-Sa 11:00-16:00'], ['Sa', 'Su 09:00-19:00'], ['Mon-Th 07:00-02:00'], ['Mon-Wed 10:00-23:00'], ['Th-Fri 11:3

In [9]:

# Remove public holidays and summer holidays
# All have PH/SH in them
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs[::-1]):
        if re.search("PH|SH", hr):
            del hours[i][-j-1]

In [10]:
# Functions for determining if statements contain days
day_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)(-(Mon|Tue|Wed|Th|Fri|Sa|Su))?"
time_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))?|off|closed"
time_range_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))|off|closed"
def contains_days(string):
    if re.search(day_re, string):
        return True
    return False

def contains_times(string):
    if re.search(time_re, string):
        return True
    return False



In [11]:
for i,hrs in enumerate(hours):
    # If a time after a comma is missing the days, then we must give it the days from the comma before 
    # e.g. Mon-Th 11:00-12:00, 13:00-15:00 =>  Mon-Th 11:00-12:00, Mon-Th 13:00-15:00
    prev_days = None
    for j, hour in enumerate(hrs):
        if contains_days(hour):
            prev_days = re.search(day_re, hour).group()
        elif prev_days:
            print("HERE", hours[i], prev_days)
            hours[i][j] = prev_days + " " + hours[i][j]
            print("HERE", hours[i], prev_days)

    
    # # If a day before a comma is missing the hours, then we must give it the days from the comma after 
    # # e.g. Mon-Th,Fri 13:00-15:00 =>  Mon-Th 13:00-15:00, Fri 13:00-15:00
    prev_hours = None
    for j, hour in enumerate(hrs[::-1]):
        if contains_times(hour):
            prev_hours = re.search(time_re, hour).group()
        elif prev_hours:
            print("HERE", hrs, prev_hours)
            hours[i][-j-1] = hours[i][-j-1] + " " + prev_hours
            print("HERE", hrs, prev_hours)


HERE ['Fri', 'Sa 11:30-15:00', '16:30-23:00'] Sa
HERE ['Fri', 'Sa 11:30-15:00', 'Sa 16:30-23:00'] Sa
HERE ['Fri', 'Sa 11:30-15:00', 'Sa 16:30-23:00'] 11:30-15:00
HERE ['Fri 11:30-15:00', 'Sa 11:30-15:00', 'Sa 16:30-23:00'] 11:30-15:00
HERE ['Su 11:00-15:00', '05:30-23:00'] Su
HERE ['Su 11:00-15:00', 'Su 05:30-23:00'] Su
HERE ['Mon', 'Wed', 'Th 11:15-22:15'] 11:15-22:15
HERE ['Mon', 'Wed 11:15-22:15', 'Th 11:15-22:15'] 11:15-22:15
HERE ['Mon', 'Wed 11:15-22:15', 'Th 11:15-22:15'] 11:15-22:15
HERE ['Mon 11:15-22:15', 'Wed 11:15-22:15', 'Th 11:15-22:15'] 11:15-22:15
HERE ['Su', 'Sa 08:00-20:00'] 08:00-20:00
HERE ['Su 08:00-20:00', 'Sa 08:00-20:00'] 08:00-20:00
HERE ['Sa', 'Su 09:00-19:00'] 09:00-19:00
HERE ['Sa 09:00-19:00', 'Su 09:00-19:00'] 09:00-19:00
HERE ['Su 11:00-13:30', '17:00-22:00'] Su
HERE ['Su 11:00-13:30', 'Su 17:00-22:00'] Su
HERE ['Fri', 'Sa 10:00-23:30'] 10:00-23:30
HERE ['Fri 10:00-23:30', 'Sa 10:00-23:30'] 10:00-23:30
HERE ['Tue-Th 09:00-15:00', '16:00-03:00'] Tue-Th
HER

In [12]:
# Now convert the closing time to a reasonable time if >23:57
# e.g. 11:00-24:00 => 11:00-00:00, 11:00-28:00 => 11:00-04:00
overtime_re = r"([01]\d|2[0-9]):([0-5]\d)(-(2[4-9]):([0-5]\d))"
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        # Find overtime hours
        search = re.search(overtime_re, hr)
        if re.search(overtime_re, hr):
            before = hr[:search.start()+6]
            # Subtract 24 from the time that's >24
            hour_reduced = int(hr[search.start() + 6:search.start()+ 8]) - 24
            after = hr[search.start()+8:]
            # Replace string with new formatted string
            hours[i][j] = f"{before}0{hour_reduced}{after}"
            

In [13]:
# Replace + symbols with -00:00 in the case where it's not already a time range 
# i.e. 11:00+ => 11:00-00:00
# But 11:00-23:00+ => 11:00-23:00
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        if "+" not in hr:
            continue
        
        replace_str = "-00:00"
        if re.search(time_range_re, hr):
            replace_str = ""

        hours[i][j] = hours[i][j].replace("+", replace_str)

In [14]:
# Remove days off from opening hours
def remove_days_off(day_str, opening_hours):
    pass

: 

In [15]:
day_time_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)?( *- *(Mon|Tue|Wed|Th|Fri|Sa|Su))? *([01]?\d|2[0-3]):([0-5]\d)( *- *([01]?\d|2[0-3]):([0-5]\d))"

no_re_count = 0
exception_count = 0
for hrs in hours:
    opening_hours = []
    for hr in hrs:
        if len(hr)>=3 and hr.endswith("off"):
            continue
        if len(hr)>=6 and hr.endswith("closed"):
            continue
        
        parse_string = re.search(day_time_re, hr)
        if not parse_string:
            print(hr, "no date")
            no_re_count += 1
            continue
        try:
            opening_hours += OpeningHours.parse(parse_string.group(), assume_type="24H").json()
        except:
            exception_count += 1

    print(hrs)
    print(opening_hours)

['Sa 07:30-15:00']
[{'day': 'saturday', 'opens': '7:30', 'closes': '15:00'}]
['Sa 15:00-04:00']
[{'day': 'saturday', 'opens': '15:00', 'closes': '4:00'}]


['Su-Mon 17:00-00:00']
[{'day': 'sunday', 'opens': '17:00', 'closes': '0:00'}]
['Mon-Fri 12:00-23:30']
[{'day': 'monday', 'opens': '12:00', 'closes': '23:30'}, {'day': 'tuesday', 'opens': '12:00', 'closes': '23:30'}, {'day': 'wednesday', 'opens': '12:00', 'closes': '23:30'}, {'day': 'thursday', 'opens': '12:00', 'closes': '23:30'}, {'day': 'friday', 'opens': '12:00', 'closes': '23:30'}]
['Th-Su 12:00-23:00']
[{'day': 'thursday', 'opens': '12:00', 'closes': '23:00'}, {'day': 'friday', 'opens': '12:00', 'closes': '23:00'}, {'day': 'saturday', 'opens': '12:00', 'closes': '23:00'}, {'day': 'sunday', 'opens': '12:00', 'closes': '23:00'}]
['Fri-Sa 10:00-02:15']
[{'day': 'friday', 'opens': '10:00', 'closes': '2:15'}, {'day': 'saturday', 'opens': '10:00', 'closes': '2:15'}]
['Su 17:00-01:00']
[{'day': 'sunday', 'opens': '17:00', 'closes': '1:00'}]
['Mon-Su 17:00-00:00']
[{'day': 'monday', 'opens': '17:00', 'closes': '0:00'}, {'day': 'tuesday', 'opens': '17:00', 'closes': '0:00'}, {'day': 'wedn

In [ ]:
for hrs in hours:
    for hr in hrs:
        if re.search("PH", hr):
            print(hrs)

In [ ]:
exception_count

1

In [ ]:
no_re_count

22